# Modeling

In [1]:
#Hello from Kaustubh
from datetime import datetime
start_time = datetime.now()

# Libraries to load
# !pip install lightgbm

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import lightgbm as lgb
import numpy as np
%matplotlib inline

print('LOAD DURATION: ', datetime.now() - start_time) # load time about 30 seconds

LOAD DURATION:  0:00:01.409526


In [ ]:
print("Kaustubh")

In [2]:
%%time
vars = pd.read_csv('vars_final_save.csv')

CPU times: user 212 ms, sys: 63.3 ms, total: 275 ms
Wall time: 277 ms


In [3]:
# use this to cap variables. For some problems it helps
Clip = 10

In [4]:
vars['Fraud'].sum()

14393

In [5]:
vars.head(10)

,record,fulladdress_count_30,ssn_firstname_count_30,homephone_count_3,ssn_dob_count_30,fulladdress_homephone_count_30,Fraud
0,1,1,1,1,1,1,0
1,2,1,1,1,1,1,1
2,3,1,1,1,1,1,0
3,4,1,1,1,1,1,0
4,5,1,1,1,1,1,0
5,6,1,1,1,1,1,0
6,7,1,1,1,1,1,0
7,8,1,1,1,1,1,0
8,9,1,1,1,1,1,0
9,10,1,1,1,1,1,0


In [6]:
vars.shape

(1000000, 7)

In [7]:
vars.describe()

,record,fulladdress_count_30,ssn_firstname_count_30,homephone_count_3,ssn_dob_count_30,fulladdress_homephone_count_30,Fraud
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000
mean,500000.500000,1.064624,1.048903,1.449296,1.046097,1.048207,0.014393
std,288675.278933,0.633831,0.499029,0.861622,0.496315,0.507682,0.119104
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,250000.750000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
50%,500000.500000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
75%,750000.250000,1.000000,1.000000,2.000000,1.000000,1.000000,0.000000
max,1000000.000000,30.000000,34.000000,32.000000,34.000000,30.000000,1.000000


In [8]:
record_save = vars['record']
Y_save = pd.DataFrame(vars.loc[:,'Fraud'])
Y_save.head()

,Fraud
0,0
1,1
2,0
3,0
4,0


## Scale and truncate field values

In [9]:
X_no_scaling = vars.drop(columns = ['record','Fraud'])
X_no_scaling.describe()

,fulladdress_count_30,ssn_firstname_count_30,homephone_count_3,ssn_dob_count_30,fulladdress_homephone_count_30
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000
mean,1.064624,1.048903,1.449296,1.046097,1.048207
std,0.633831,0.499029,0.861622,0.496315,0.507682
min,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,2.000000,1.000000,1.000000
max,30.000000,34.000000,32.000000,34.000000,30.000000


In [10]:
X = (X_no_scaling - X_no_scaling.mean()) / X_no_scaling.std()

In [11]:
# push in any outlier values
cols = X.columns
X.loc[:,cols] = X[cols].clip(upper=Clip)
X.loc[:,cols] = X[cols].clip(lower=-1*Clip)
# X = (X_no_scaling - X_no_scaling.mean()) / X_no_scaling.std()
X.describe()

,fulladdress_count_30,ssn_firstname_count_30,homephone_count_3,ssn_dob_count_30,fulladdress_homephone_count_30
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000
mean,-0.018176,-0.017457,-0.005814,-0.017649,-0.018020
std,0.631721,0.579481,0.910935,0.571302,0.576299
min,-0.101958,-0.097996,-0.521454,-0.092879,-0.094955
25%,-0.101958,-0.097996,-0.521454,-0.092879,-0.094955
50%,-0.101958,-0.097996,-0.521454,-0.092879,-0.094955
75%,-0.101958,-0.097996,0.639148,-0.092879,-0.094955
max,10.000000,10.000000,10.000000,10.000000,10.000000


In [12]:
# separate data into modeling (traintest) and out of time
X_trntst = X[0:833507]
Y_trntst = Y_save[0:833507]
X_oot = X[833507:]
Y_oot = Y_save[833507:]

In [13]:
niter = 0
nitermax = 3
FDR3 = pd.DataFrame(np.zeros((nitermax,3)), columns=('trn', 'tst', 'oot'))
X_oot_orig = X_oot.copy()
pd.options.mode.chained_assignment = None  # default='warn'

In [14]:
%%time
# Logistic regression

for niter in range(nitermax):    
    X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst, Y_trntst, test_size = .3)

    model = LogisticRegression()
 
    X_oot = X_oot_orig.copy()
    X_trn_save = X_trn.copy()
    Y_trn_save = Y_trn.copy()

    model.fit(X_trn, Y_trn.values.ravel())   

    predictions = model.predict_proba(X_trn_save)[:,1]
    X_trn['predicted'] = predictions
    X_trn['Fraud'] = Y_trn_save['Fraud']
    topRows = int(round(X_trn.shape[0]*0.03))
    temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'Fraud'])

    predictions = model.predict_proba(X_tst)[:,1]
    X_tst['predicted']=predictions
    X_tst['Fraud'] = Y_tst['Fraud']
    topRows = int(round(X_tst.shape[0]*0.03))
    temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'Fraud'])

    predictions = model.predict_proba(X_oot)[:,1]
    X_oot['predicted']=predictions
    X_oot['Fraud'] = Y_oot['Fraud']
    topRows = int(round(X_oot.shape[0]*0.03))
    temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'Fraud'])
    print(niter, FDR3.loc[niter, 'trn'],FDR3.loc[niter, 'tst'],FDR3.loc[niter, 'oot'])
    
print(FDR3.mean())

0 0.5001791472590469 0.5044028618602091 0.48575020955574183
1 0.5044499822000712 0.49273743016759775 0.48575020955574183
2 0.5032687507428979 0.49638286032276013 0.48575020955574183
trn    0.502633
tst    0.497841
oot    0.485750
dtype: float64
CPU times: user 2.43 s, sys: 67.1 ms, total: 2.5 s
Wall time: 2.51 s


## Models. You can comment out any of these cells and just explore one model type. You can also just rerun that single cell multiple times as you explore different model hyperparameters.

In [15]:
%%time
# Single DT

for niter in range(nitermax):    
    X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst, Y_trntst, test_size = .3)

    model = DecisionTreeClassifier()
 
    X_oot = X_oot_orig.copy()
    X_trn_save = X_trn.copy()
    Y_trn_save = Y_trn.copy()

    model.fit(X_trn, Y_trn.values.ravel())   

    predictions = model.predict_proba(X_trn_save)[:,1]
    X_trn['predicted'] = predictions
    X_trn['Fraud'] = Y_trn_save['Fraud']
    topRows = int(round(X_trn.shape[0]*0.03))
    temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'Fraud'])

    predictions = model.predict_proba(X_tst)[:,1]
    X_tst['predicted']=predictions
    X_tst['Fraud'] = Y_tst['Fraud']
    topRows = int(round(X_tst.shape[0]*0.03))
    temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'Fraud'])

    predictions = model.predict_proba(X_oot)[:,1]
    X_oot['predicted']=predictions
    X_oot['Fraud'] = Y_oot['Fraud']
    topRows = int(round(X_oot.shape[0]*0.03))
    temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'Fraud'])
    print(niter, FDR3.loc[niter, 'trn'],FDR3.loc[niter, 'tst'],FDR3.loc[niter, 'oot'])
    
print(FDR3.mean())

0 0.5571599045346062 0.5563826854149435 0.5339480301760269
1 0.564090368608799 0.5401723658604393 0.5364626990779547
2 0.5559366130558183 0.5578710222472543 0.5356244761106455
trn    0.559062
tst    0.551475
oot    0.535345
dtype: float64
CPU times: user 1.41 s, sys: 23.7 ms, total: 1.43 s
Wall time: 1.43 s


In [16]:
%%time
# RF

for niter in range(nitermax):    
    X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst, Y_trntst, test_size = .3)

    model = RandomForestClassifier()
    X_oot = X_oot_orig.copy()
    X_trn_save = X_trn.copy()
    Y_trn_save = Y_trn.copy()

    model.fit(X_trn, Y_trn.values.ravel())   

    predictions = model.predict_proba(X_trn_save)[:,1]
    X_trn['predicted'] = predictions
    X_trn['Fraud'] = Y_trn_save['Fraud']
    topRows = int(round(X_trn.shape[0]*0.03))
    temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'Fraud'])

    predictions = model.predict_proba(X_tst)[:,1]
    X_tst['predicted']=predictions
    X_tst['Fraud'] = Y_tst['Fraud']
    topRows = int(round(X_tst.shape[0]*0.03))
    temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'Fraud'])

    predictions = model.predict_proba(X_oot)[:,1]
    X_oot['predicted']=predictions
    X_oot['Fraud'] = Y_oot['Fraud']
    topRows = int(round(X_oot.shape[0]*0.03))
    temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'Fraud'])
    print(niter, FDR3.loc[niter, 'trn'],FDR3.loc[niter, 'tst'],FDR3.loc[niter, 'oot'])
    
print(FDR3.mean())

0 0.5573282533634957 0.5529379157427938 0.5360435875943
1 0.5576529998797644 0.5547425474254742 0.537300922045264
2 0.5618004286734937 0.5455804932114159 0.537300922045264
trn    0.558927
tst    0.551087
oot    0.536882
dtype: float64
CPU times: user 56.7 s, sys: 95.1 ms, total: 56.8 s
Wall time: 56.9 s


In [17]:
%%time
# BT

for niter in range(nitermax):    
    X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst, Y_trntst, test_size = .3)

    model = lgb.LGBMClassifier()

    X_oot = X_oot_orig.copy()
    X_trn_save = X_trn.copy()
    Y_trn_save = Y_trn.copy()

    model.fit(X_trn, Y_trn.values.ravel())   

    predictions = model.predict_proba(X_trn_save)[:,1]
    X_trn['predicted'] = predictions
    X_trn['Fraud'] = Y_trn_save['Fraud']
    topRows = int(round(X_trn.shape[0]*0.03))
    temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'Fraud'])

    predictions = model.predict_proba(X_tst)[:,1]
    X_tst['predicted']=predictions
    X_tst['Fraud'] = Y_tst['Fraud']
    topRows = int(round(X_tst.shape[0]*0.03))
    temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'Fraud'])

    predictions = model.predict_proba(X_oot)[:,1]
    X_oot['predicted']=predictions
    X_oot['Fraud'] = Y_oot['Fraud']
    topRows = int(round(X_oot.shape[0]*0.03))
    temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'Fraud'])
    print(niter, FDR3.loc[niter, 'trn'],FDR3.loc[niter, 'tst'],FDR3.loc[niter, 'oot'])
    
print(FDR3.mean())

0 0.5517698049602697 0.5709267765468792 0.5364626990779547
1 0.5563748810656518 0.5568213392609058 0.5364626990779547
2 0.5559900462140064 0.5608183856502242 0.5364626990779547
trn    0.554712
tst    0.562856
oot    0.536463
dtype: float64
CPU times: user 18.2 s, sys: 111 ms, total: 18.3 s
Wall time: 5.25 s


In [18]:
%%time
# NN

for niter in range(nitermax):    
    X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst, Y_trntst, test_size = .3)

    model = MLPClassifier()

    X_oot = X_oot_orig.copy()
    X_trn_save = X_trn.copy()
    Y_trn_save = Y_trn.copy()

    model.fit(X_trn, Y_trn.values.ravel())   

    predictions = model.predict_proba(X_trn_save)[:,1]
    X_trn['predicted'] = predictions
    X_trn['Fraud'] = Y_trn_save['Fraud']
    topRows = int(round(X_trn.shape[0]*0.03))
    temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'Fraud'])

    predictions = model.predict_proba(X_tst)[:,1]
    X_tst['predicted']=predictions
    X_tst['Fraud'] = Y_tst['Fraud']
    topRows = int(round(X_tst.shape[0]*0.03))
    temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'Fraud'])

    predictions = model.predict_proba(X_oot)[:,1]
    X_oot['predicted']=predictions
    X_oot['Fraud'] = Y_oot['Fraud']
    topRows = int(round(X_oot.shape[0]*0.03))
    temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'Fraud'])
    print(niter, FDR3.loc[niter, 'trn'],FDR3.loc[niter, 'tst'],FDR3.loc[niter, 'oot'])
    
print(FDR3.mean())

0 0.5583615273078782 0.5505226480836237 0.5368818105616094
1 0.5500774085983089 0.5695290858725762 0.5352053646269908
2 0.5583666745116498 0.5500142490738102 0.5368818105616094
trn    0.555602
tst    0.556689
oot    0.536323
dtype: float64
CPU times: user 2min 39s, sys: 445 ms, total: 2min 40s
Wall time: 2min 40s


In [19]:
print('duration: ', datetime.now() - start_time)

duration:  0:03:48.900114
